<a href="https://colab.research.google.com/github/hyeokjun0804/ML-DL/blob/main/10_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#교차 검증과 그리드 서치
###IT융합자율학부 201814076 권혁준

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy() # to.numpy()로 2차원 배열로 만들어줌
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target)) #훈련 세트
print(dt.score(val_input, val_target)) #테스트 세트

0.9971133028626413
0.864423076923077


##교차 검증
- 훈련 세트에서 검증 세트를 분할하여 평가하는 과정을 반복하는 방식
 - 반복하여 얻는 검증 점수를 평균하여 최종 검증 점수를 계산
- estimator: 평가할 모델 객체
- x: 훈련 세트
- y: 훈련 세트의 타겟
- cv=None: 폴드 수 또는 교차 검증 생성기
- 교차 검증의 최종 점수는 test_score 키의 점수의 평균으로 계산

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01510715, 0.01056004, 0.01104617, 0.01127267, 0.01052809]), 'score_time': array([0.0021193 , 0.00192738, 0.00199103, 0.00199294, 0.00190115]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # splitter(분할기):교차 검증에서 폴드를 어떻게 나눌지 결정하는 객체
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


##하이퍼마라미터 튜닝
- 모델이 학습할 수 없어서 사용자가 지정해야 하는 파라미터
- 머신러닝 라이브러리를 사용할 때 클래스난 메서드의 매개변수로 표현
- 대체로 하이퍼파라미터 간에 종속성이 존재
 - 결정 트리에서 min_samples_split 매개변수의 값이 바꾸면 max_depth의 최적값도 달라짐
- 다수의 하이퍼파라미터가 있는 경우 여러 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 함

##그리드 서치 GridSearch
- 하이퍼파라미터와 교차 검증을 한 번에 수행
- 매개변수
 - estimator: 평가할 모델 객체
 - param_grid: 모델의 매개변수를 키로 하고, 설정할 인자를 값으로 하는 딕셔너리 객체
 - n_jobs=1: 병렬 처리에 사용할 CPU 코어 수(-1: all cores)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


##랜덤 서치

In [ ]:
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0, 10) # 정숫값의 균등 분포 객체
rgen.rvs(10)

array([2, 2, 6, 4, 9, 5, 3, 5, 6, 8])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 87, 117,  94, 102, 112, 100,  96,  88, 101, 103]))

In [ ]:
ugen = uniform(0,1) # 실숫값의 균등 분포 객체
ugen.rvs(10)

array([0.23104051, 0.83332653, 0.22877422, 0.2718403 , 0.39675918,
       0.51711244, 0.27060808, 0.65993825, 0.69632327, 0.51026889])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': range(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(20, 50),
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x79f330d36080>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79f330d34d90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79f330d34cd0>},
                   random_state=42)

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
